In [10]:
from math import *
import random

class five_in_line:
    def __init__(self,size):
        self.playerJustMoved = 2  # At the root pretend the player just moved is player 2 - player 1 has the first move
        self.board = []  # 0 = empty, 1 = player 1, 2 = player 2
        self.size = size
        for y in range(size):  # initialize all board position as empty in the board
            self.board.append([0] * size)      
    def Clone(self):
        """ Create a deep clone of this game state.       
        """
        size=self.size
        st = five_in_line(size)
        st.playerJustMoved = self.playerJustMoved
        return st
   
   
  
    def IsFiveInLine(self, x, y):
        """ Speeds up GetMoves by only considering which are connected five in line of the playerjm.       
        """
        # THis is the part you need to implement to complete the five_in_line game program
        # you need to specify the concept to detect whether a board consists of five in a line in one player ot
        # other to decide which player is the winner
        # The predicat function is called by GetResult(self,playerJustMoved)
        target = self.board[x][y]

        # |
        count = 1
        for i in range(1, y + 1):
            if self.board[x][y - i] == target:
                count = count + 1
            else:
                break
        for i in range(1, self.size - y):
            if self.board[x][y + i] == target:
                count = count + 1
            else:
                break
        if count >= 5:
            return True

        # -
        count = 1
        for i in range(1, x + 1):
            if self.board[x - i][y] == target:
                count = count + 1
            else:
                break
        for i in range(1, self.size - x):
            if self.board[x + i][y] == target:
                count = count + 1
            else:
                break
        if count >= 5:
            return True

        # /
        count = 1
        for i in range(1, 5):
            if x - i < 0 or y - i < 0:
                break
            if self.board[x - i][y - i] == target:
                count = count + 1
            else:
                break
        for i in range(1, 5):
            if x + i >= self.size or y + i >= self.size:
                break
            if self.board[x + i][y + i] == target:
                count = count + 1
            else:
                break
        if count >= 5:
            return True

        # \
        count = 1
        for i in range(1, 5):
            if x - i < 0 or y + i >= self.size:
                break
            if self.board[x - i][y + i] == target:
                count = count + 1
            else:
                break
        for i in range(1, 5):
            if x + i >= self.size or y - i < 0:
                break
            if self.board[x + i][y - i] == target:
                count = count + 1
            else:
                break
        if count >= 5:
            return True
        
        return False
        
    def IsOnBoard(self, x, y):
        return x >= 0 and x < self.size and y >= 0 and y < self.size
    
    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerJustMoved.
        """       
        (x, y) = (move[0], move[1])
        assert x == int(x) and y == int(y) and self.IsOnBoard(x, y) and self.board[x][y] == 0 # if not satisfied report error
        m = self.GetMoves()      
             
        self.playerJustMoved = 3 - self.playerJustMoved # update the player
        self.board[x][y] = self.playerJustMoved 

    def GetMoves(self):
        """ Get all possible moves from this state.
        """
        return [(x, y) for x in range(self.size) for y in range(self.size) if
                self.IsOnBoard(x,y) and self.board[x][y] == 0 ]

    def GetResult(self,playerjm):
        for x in range(self.size): 
            for y in range(self.size):
                if self.board[x][y] == playerjm and self.IsFiveInLine(x, y):
                    return 1.0  # playerjm wins when there is five in line                  
                elif self.board[x][y] == 3 - playerjm and self.IsFiveInLine(x, y):                       
                    return 0.0  # playerjm looses when the opponent has five in line       
        return 0.5  # draw when both run out of empty position to play and nobody wins

    def __repr__(self):
        """ Don't need this - but good style.
        """
        pass
    
class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """

    def __init__(self, move=None, parent=None, state=None):
        self.move = move  # the move that got us to this node - "None" for the root node
        self.parentNode = parent  # "None" for the root node
        self.childNodes = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves()  # future child nodes
        self.playerJustMoved = state.playerJustMoved  # the only part of the state that the Node needs lat...
    
    def UCTSelectChild(self):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation.
        """
        s = sorted(self.childNodes, key=lambda c: c.wins / c.visits + sqrt(2 * log(self.visits) / c.visits))[-1]
        return s

    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        n = Node(move=m, parent=self, state=s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n

    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += result

    def __repr__(self):
        return "[M:" + str(self.move) + " W/V:" + str(self.wins) + "/" + str(self.visits) + " U:" + str(self.untriedMoves) + "]"

    def TreeToString(self, indent):
        s = self.IndentString(indent) + str(self)
        for c in self.childNodes:
            s += c.TreeToString(indent + 1)
        return s

    def IndentString(self, indent):
        s = "\n"
        for i in range(1, indent + 1):
            s += "| "
        return s

    def ChildrenToString(self):
        s = ""
        for c in self.childNodes:
            s += str(c) + "\n"
        return s

def UCT(rootstate, itermax, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
        Assumes 2 alternating players (player 1 starts), with game results in the range [0.0, 1.0]."""
    rootnode = Node(state=rootstate)

    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone()

        # Select
        while node.untriedMoves == [] and node.childNodes != []:  # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)

        # Expand
        if node.untriedMoves != []:  # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves)
            state.DoMove(m)
            node = node.AddChild(m, state)  # add child and descend tree

        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []:  # while state is non-terminal
 #           print("****"+ str(state.GetMoves()))
            state.DoMove(random.choice(state.GetMoves()))

        # Backpropagate
        while node != None:  # backpropagate from the expanded node and work back to the root node
            
            node.Update(state.GetResult(node.playerJustMoved))  # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode

    # Output some information about the tree - can be omitted
    if (verbose):
        print(rootnode.TreeToString(0))
    else:
 #       print(rootnode.ChildrenToString())
        print("*")
    return sorted(rootnode.childNodes, key=lambda c: c.visits)[-1].move  # return the move that was most visited
    
def UCTPlayGame():
    """ Play a sample game between two UCT players where each player gets a different number
        of UCT iterations (= simulations = tree nodes).
    """
    state = five_in_line(7)    
    count=0
    while (state.GetMoves() != []): 
 #       print("player", state.playerJustMoved, str(state.GetMoves()))
        if state.playerJustMoved == 1:
            m = UCT(rootstate = state, itermax = 100, verbose = False)  # play with values for itermax and verbose = True
            count +=1
        else:
            m = UCT(rootstate = state, itermax = 100, verbose = False) # you could implement a weak play or human player here
            count +=1
            # what if player 2 is a random player
            # node= Node(_, _, state)
            # m  = random.choice(node.untriedMoves)?
        
        # you could implement heuristic such as live_three, live_four_missing_one, and double_three pattern concepts so that it 
        # can be evaluated if an empty_move can be detected to have the pattern and be combined with the
        # UCT search to determine the best move. ie If these concepts appear, you allow the priority to form the patterns and
        # if they are not avialble/ are more than one you end-up UCT search
        state.DoMove(m)
        print("play#", count, ", Player " + str(state.playerJustMoved)+" Best Move: " + str(m))
       
        s=""
        for y in range(state.size - 1, -1, -1):  # start from large y decrement 1 to 0
            for x in range(state.size):                
                    s += str(state.board[x][y])
            s += "\n"
        print(s)
      
        if state.GetResult(state.playerJustMoved) == 1.0:
            print("Player " + str(state.playerJustMoved) + " wins!")
            break
        elif state.GetResult(3 - state.playerJustMoved) == 0.0:
            print("Player " + str(3 - state.playerJustMoved) + " wins!")
            break
        else: print("No one wins yet")

if __name__ == "__main__":
    """ Play a single game to the end using UCT for both players. 
    """
    UCTPlayGame()

    
    

*
(3, 2)
play# 1 , Player 1 Best Move: (3, 2)
0000000
0000000
0000000
0000000
0001000
0000000
0000000

No one wins yet
*
(0, 3)
play# 2 , Player 2 Best Move: (0, 3)
0000000
0000000
0000000
2000000
0001000
0000000
0000000

No one wins yet
*
(2, 5)
play# 3 , Player 1 Best Move: (2, 5)
0000000
0010000
0000000
2000000
0001000
0000000
0000000

No one wins yet
*
(1, 5)
play# 4 , Player 2 Best Move: (1, 5)
0000000
0210000
0000000
2000000
0001000
0000000
0000000

No one wins yet
*
(1, 1)
play# 5 , Player 1 Best Move: (1, 1)
0000000
0210000
0000000
2000000
0001000
0100000
0000000

No one wins yet
*
(6, 6)
play# 6 , Player 2 Best Move: (6, 6)
0000002
0210000
0000000
2000000
0001000
0100000
0000000

No one wins yet
*
(0, 5)
play# 7 , Player 1 Best Move: (0, 5)
0000002
1210000
0000000
2000000
0001000
0100000
0000000

No one wins yet
*
(1, 0)
play# 8 , Player 2 Best Move: (1, 0)
0000002
1210000
0000000
2000000
0001000
0100000
0200000

No one wins yet
*
(4, 6)
play# 9 , Player 1 Best Move: (4, 6)
00